### More IMDB Data

In [3]:
import pandas as pd

In [4]:
name_basics_df = pd.read_csv('../zippedData/extra.imdb/name.basics.tsv/data.tsv.gz', 
                             delimiter='\t', usecols=['nconst', 'primaryName'])

In [6]:
name_basics_df.head(2)

,nconst,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall


In [5]:
title_akas_df = pd.read_csv('../zippedData/extra.imdb/title.akas.tsv/data.tsv.gz', 
                            delimiter='\t', usecols=['titleId', 'ordering', 'title'])

In [7]:
title_akas_df.head(2)

,titleId,ordering,title
0,tt0000001,1,Карменсіта
1,tt0000001,2,Carmencita


In [27]:
title_basics_df = pd.read_csv('../zippedData/extra.imdb/title.basics.tsv/data.tsv.gz', 
                              delimiter='\t', usecols=['tconst', 'titleType', 'primaryTitle', 
                              'originalTitle', 'startYear', 'runtimeMinutes', 'genres'])

In [9]:
title_basics_df.head(2)

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,1894,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,1892,5,"Animation,Short"


In [10]:
title_principals_df = pd.read_csv('../zippedData/extra.imdb/title.principals.tsv/data.tsv.gz', 
                              delimiter='\t', usecols=['tconst', 'ordering', 'nconst', 'category'])

In [11]:
title_principals_df.head(2)

,tconst,ordering,nconst,category
0,tt0000001,1,nm1588970,self
1,tt0000001,2,nm0005690,director


In [12]:
title_ratings_df = pd.read_csv('../zippedData/extra.imdb/title.ratings.tsv/data.tsv.gz', 
                              delimiter='\t', usecols=['tconst', 'averageRating', 'numVotes'])

In [13]:
title_ratings_df.head(2)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1990
1,tt0000002,5.8,265


In [28]:
title_basics_df['startYear'] = pd.to_datetime(title_basics_df['startYear'], errors='coerce', format='%Y').dt.year

In [33]:
movies_df = title_basics_df[(title_basics_df['startYear'] >= 2000) & (title_basics_df['titleType'] == 'movie')]

In [39]:
movies_df = movies_df.merge(title_ratings_df, on='tconst', how='left')

In [42]:
movies_df = movies_df.merge(title_principals_df, on='tconst', how='left')

In [43]:
movies_df

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,ordering,nconst,category
0,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,2019.0,\N,"Action,Crime",NaN,NaN,10.0,nm0459029,actor
1,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,2019.0,\N,"Action,Crime",NaN,NaN,1.0,nm0681726,actor
2,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,2019.0,\N,"Action,Crime",NaN,NaN,2.0,nm0692612,actress
3,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,2019.0,\N,"Action,Crime",NaN,NaN,3.0,nm0726256,actor
4,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,2019.0,\N,"Action,Crime",NaN,NaN,4.0,nm0776458,actor
...,...,...,...,...,...,...,...,...,...,...,...,...
2261268,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013.0,49,Documentary,NaN,NaN,5.0,nm9272490,director
2261269,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013.0,49,Documentary,NaN,NaN,6.0,nm8349149,director
2261270,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013.0,49,Documentary,NaN,NaN,7.0,nm9272489,cinematographer
2261271,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013.0,49,Documentary,NaN,NaN,8.0,nm10538638,cinematographer
